<img src="https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/http://coursera-university-assets.s3.amazonaws.com/4a/cb36835ae3421187080898a7ecc11d/Google-G_360x360.png?auto=format%2Ccompress&dpr=1&w=56px&h=56px&auto=format%2Ccompress&dpr=1&w=&h=">

## Google Data Analytics Capstone

<div style="clear: both"></div>
<hr style="width:420px; margin: auto; float: left" />

<div style="clear: both"></div>
<div style="clear: both"></div>
<hr style="width:420px; margin: auto; float: left" />

<div style="clear: both"></div>




<h3>Abdulrahman Alazwari</h3>
<a href="https://github.com/azwri/">
    <img src='logos/github-logo.png' width='30'  />
</a><span> &nbsp;&nbsp;&nbsp; </span><a href="https://www.linkedin.com/in/abdulrahman-alazwari/">
    <img src='logos/linkedin-logo.png' width='30' />
</a><span> &nbsp;&nbsp;&nbsp; </span><a href="https://twitter.com/Al_Azwari">
    <img src='logos/twitter-logo.png' width='30' />
</a><span> &nbsp;&nbsp;&nbsp; </span><a href = "mailto: aazwri@gmail.com">
    <img src='logos/gmail-logo.png' width='40' />
</a>
<br><br>
<div style="clear: both"></div>
<hr style="width:220px; margin: auto; float: left" />

<div style="clear: both"></div>


## Import the libraries
> if error download the below<br>
>> !pip install arabic-reshaper<br>
>> !pip install python-bidi<br>
>> !pip install folium<br>
>> !pip install wordcloud<br>
>> !pi install pillow<br>

In [1]:
# !pip install arabic-reshaper
# !pip install python-bidi
# !pip install folium
# !pip install wordcloud
# !pip install pillow

import os
import pandas as pd
import numpy as np
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pylab as plt
import folium
import arabic_reshaper
from bidi.algorithm import get_display

%matplotlib inline
# disable chained assignments
pd.options.mode.chained_assignment = None 

ModuleNotFoundError: No module named 'wordcloud'

## Gather Data

Download the CSV file from [source link](https://datasource.kapsarc.org/explore/dataset/saudi-arabia-coronavirus-disease-covid-19-situation/export/?sort=-daily_accumulative&dataChart=%3D&disjunctive.daily_accumulative=&flg=ar&disjunctive.city&disjunctive.daily_cumulative&disjunctive.indicator&disjunctive.event&disjunctive.region&disjunctive.city_en&disjunctive.region_en).<br/>
* The csv file is more than 19 GB right now, June 2021.<br/>
* First download the file and read it by `pandas.read_csv` method and pass argument `;` to `sep` parameter, because they format the csv file by `;`.<br/>
* Second use a tricky way to lower the size of CSV file to be about 2 GB:
> Save the file by `to_csv` method and pass argument `gzip` extension to `compression` parameter to compress the file and lower the size, and importantly pass `False` to index parameter to exclude the index column.<br/>
> For memory reason delete the orginal CSV file by `os.remove` method.<br/>
> Read the new file by `pandas.read_csv` method and pass argument `gzip` to `compression` parameter.<br/>

In [ ]:
# csv_file = pd.read_csv('saudi-arabia-coronavirus-disease-covid-19-situation.csv', sep=';')
# csv_file.to_csv('sa-covid-19-v1.csv', compression='gzip', index=False)
# os.remove('saudi-arabia-coronavirus-disease-covid-19-situation.csv')

**Read CSV file and assign it to `df`os.remove variable**

In [ ]:
df = pd.read_csv('sa-covid-19-v1.csv', compression='gzip')

## Assess Data

> Use `head` method to have a look on the first 5 observations.<br/>
> Use `columns.values` property to return the columns names.<br/>
> Use `info` meethod to see missing values and data types of each feature.<br/>
> Also we can use `dtype` property to see the data types.<br/>
> Summary statistics on numeric values by using `describe` method.<br/>

In [ ]:
df.head()

In [ ]:
df.columns.values

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.describe()

## Clean Data
> Make a data frame copy to manipulate the data<br>
> Filter on `daliy` cases from `Daily / Cumulative` column<br>
> Drop attributes, which we do not need `Daily / Cumulative` and `Event`<br>
> Convert columns names to lower case to use it as attribute when call it<br>
> Rename last attribute `cases (person)` to `cases`<br>
> Drop unnecessary record (“Total”)
> Assign total_record variable to `Total` record and total_record_index variable to indexes of total_record<br>
> Check missing values, and drip them or change them<br>
> Chenge data type of case attribute and chenge data type of date attribute<br>

In [ ]:
# make copy from the orginal dataframe
df_copy = df.copy()

In [ ]:
df_copy.head()

In [ ]:
# Filter on `daliy` cases from `Daily / Cumulative` column
df_copy = df[df_copy['Daily / Cumulative'] == 'Daily']

In [ ]:
# Drop attributes, which we do not need `Daily / Cumulative` and `Event`
df_copy.drop(['Daily / Cumulative', 'Event'], axis=1, inplace=True)

In [ ]:
# check the columns after drop
df_copy.columns.values

In [ ]:
# convert columns names to lower case to use it as attribute when call it
# rename last attribute `cases (person)` to `cases`
df_copy.rename(columns=lambda col: col.lower(), inplace=True)
df_copy.rename(columns={'cases (person)': 'cases'}, inplace=True)

In [ ]:
# check the columns after convert to lower case and rename `cases (person)` to `case`
df_copy.columns.values

In [ ]:
# drop unnecessary record (“Total”)
# assign total_record variable to `Total` record and total_record_index variable to indexes of total_record
# print(how many record with total value)
total_record = df_copy[df_copy.city == 'Total']
total_record_index = total_record.index.tolist()
len(total_record_index)

In [ ]:
# drop the `Total` record
df_copy.drop(total_record_index, axis=0, inplace=True)

In [ ]:
# check the data frame after drop `Total` records
df_copy[df_copy.city == 'Total']

In [ ]:
# check missing values
df_copy.isnull().sum()

In [ ]:
# check format
df_copy.dtypes

In [ ]:
# chenge data type of case attribute and chenge data type of date attribute, then check format again
df_copy.cases.astype('int')
df_copy['date'] = pd.to_datetime(df_copy['date'])
df_copy.dtypes

## Save a New Data Frame after Changing, one normal and the other is compressed

In [ ]:
df_copy.to_csv('sa-covid-19-v2.csv', index=False)
df_copy.to_csv('sa-covid-19-v2-compressed.csv', index=False, compression='gzip')

## Descriptive Statistics: Indicator vs. Cases, then:
> ### Split by indicator
>> ### make four data frames for each


In [ ]:
df_copy.indicator.value_counts()

In [ ]:
cases = df_copy.query("indicator == 'Cases'").reset_index(drop=True)
recoveries = df_copy.query("indicator == 'Recoveries'").reset_index(drop=True)
active = df_copy.query("indicator == 'Active'").reset_index(drop=True)
mortalities = df_copy.query("indicator == 'Mortalities'").reset_index(drop=True)

In [ ]:
cases.head()

## Grouping Data:
> ### Indicator vs.Region vs. Cases

In [ ]:
df_copy.groupby(['indicator', 'region']).sum()

# Binning:
> I choose this websit to make levels as below
https://www.cdc.gov/coronavirus/2019-ncov/travelers/how-level-is-determined.html

<img src="logos/levels.png" />


### make function for binning

In [ ]:
# this funcation take one parameter then return level depending on the conditions below
def level(val):
    if val < 49:
        return 'LOW'
    elif val < 99:
        return 'MODERATE'
    elif val <= 500:
        return 'HIGH'
    else:
        return 'VERY HIGH'

In [ ]:
# Add new columns to the cases data frame for binning
# Below, pass the function level to apply function
# apply function will put bin for each record

In [ ]:
cases['cases_binned'] = cases.cases.apply(level)

In [ ]:
cases.head()

# Visualization

In [ ]:
cases.cases_binned.value_counts()

In [ ]:
# binning for cases
cases.cases_binned.value_counts().plot(kind='pie', figsize=(10, 7))
plt.title("Daliy Cases Levels")
plt.show()

### Line Plot
> #### Use line plot to draw a line for every week and only “Cases” Parameters: Every Week + Indicator: Cases

In [ ]:
cases['week'] = cases['date'].map(lambda x: x.isocalendar()[1])
cases['month'] = pd.DatetimeIndex(cases['date']).month
cases['year'] = cases['date'].map(lambda x: x.isocalendar()[0])

In [ ]:

cases[cases.year == 2020].groupby('week').sum()['cases'].sort_index().plot(figsize=(20, 5))
plt.yticks([ 1000, 5000, 10000, 15000, 20000, 25000, 30000], [ '2.5K', '5K', '10K', '15K', '20K', '25K', '30K'])
plt.xticks([0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55], 
           ['Week 1', 'Week 5', 'Week 10', 'Week 15', 'Week 20', 'Week 25', 'Week 30', 'Week 35', 'Week 40', 'Week 45', 'Week 50','Year 2021'])
plt.title("COVID-19 Weekly Cases - 2020")
plt.xlabel("Week Number")
plt.ylabel("Number Of Cases")
plt.show()

In [ ]:
cases[(cases.year == 2020) & (cases.month.notna())].groupby('month').sum()['cases'].sort_index().plot(figsize=(20, 5))
plt.yticks([0, 20000, 40000, 60000, 80000, 100000, 120000], ['0', '20K', '40K', '60K', '80K', '100K', '120K'])
plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
           ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov','Dec'])
plt.title("COVID-19 Monthly Cases - 2020")
plt.xlabel("Month")
plt.ylabel("Number Of Cases")
plt.show()

In [ ]:

cases[cases.year == 2021].groupby('week').sum()['cases'].sort_index().plot(figsize=(20, 5))
plt.yticks([0, 1000, 5000, 10000, 15000, 20000, 25000, 30000], ['0', '1K', '5K', '10K', '15K', '20K', '25K', '30K'])
plt.xticks([0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55], 
           ['Week 1', 'Week 5', 'Week 10', 'Week 15', 'Week 20', 'Week 25', 'Week 30', 'Week 35', 'Week 40', 'Week 45', 'Week 50','Year 2022'])
plt.title("COVID-19 Weekly Cases - 2021")
plt.xlabel("Week Number")
plt.ylabel("Number Of Cases")
plt.show()

In [ ]:
cases[(cases.year == 2021) & (cases.month.notna())].groupby('month').sum()['cases'].sort_index().plot(figsize=(20, 5))
plt.yticks([0, 20000, 40000, 60000, 80000, 100000, 120000], ['0', '20K', '40K', '60K', '80K', '100K', '120K'])
plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
           ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov','Dec'])
plt.title("COVID-19 Monthly Cases - 2021")
plt.xlabel("Month")
plt.ylabel("Number Of Cases")
plt.show()

### Area Plot
> #### Use area plot to draw a Top 5: Total Cases Cities

In [ ]:
top_5_city = cases.groupby('city').sum()[['cases']].sort_values(by='cases', ascending=False)[:5].T

In [ ]:
top_5_city = top_5_city.append([top_5_city]*5,ignore_index=True)

In [ ]:
top_5_city.plot(kind='area', figsize=(20, 10))
plt.title('Top 5 Cites Cases')
plt.show()

### Pie Chart
> #### Use Pie chart to draw total cases in regions.

In [ ]:
total_cases = cases.groupby('region').sum()[['cases']]
total_cases.plot(kind='pie', y='cases', figsize=(20, 10), legend=False)
plt.title('Regions VS Cases')
plt.show()

### Visualizing Geospatial Data
> #### Use Folium Library to draw total cases in regions.

In [ ]:
# read `regions.json` file, contains jps coordinates
gps_json = pd.read_json('regions.json')

In [ ]:
# assign `region_cases_en` to regions from te data frame
region_cases_en = cases.groupby('region').sum()[['cases']].to_dict()['cases']

# create a dictionary `regions_ar` to Arabic region names
regions_ar = {
    "Eastern Region": "المنطقة الشرقية",
    "Ar Riyad":"منطقة الرياض",
    "Makkah Al Mukarramah":"منطقة مكة المكرمة",
    "Aseer":"منطقة عسير",
    "Al Qaseem":"منطقة القصيم",
    "Jazan":"منطقة جازان",
    "Al Madinah Al Munawwarah":"منطقة المدينة المنورة",
    "Al Bahah":"منطقة الباحة",
    "Hail":"منطقة حائل",
    "Tabuk":"منطقة تبوك",
    "Najran":"منطقة نجران",
    "Al Jawf":"منطقة الجوف",
    "Northern Borders":"منطقة الحدود الشمالية",
}



In [ ]:
# create a dictionary `region_cases_ar` to Arabic & English region names
region_cases_ar = {}
for i in regions_ar:
    region_cases_ar[i] = regions_ar[i]

    
# create a dictionary `jps_ar` contains Arabic region name and jps coordinates
jps_ar = gps_json[['name_ar', 'center']].set_index('name_ar').T.to_dict()

region_ar_en = { k:v for k,v in zip(region_cases_ar.values(), region_cases_ar.keys()) }

# last variable for Arabic region and jps coordinates
center_gps = {}
for i in region_ar_en:
    center_gps[i] = (jps_ar[i]['center'])


In [ ]:
region_coordinates = {}
for i in total_cases.to_dict()['cases'].items():
    region_coordinates[i[0]] = [i[0], regions_ar[i[0]], center_gps[regions_ar[i[0]]][0], center_gps[regions_ar[i[0]]][1], region_cases_en[i[0]]]

In [ ]:
gps_data_frame = pd.DataFrame(region_coordinates).T

In [ ]:
# function for color the icon on map

def color(c):
    if c < 10000:
        return 'green'
    elif c < 50000:
        return 'lightgreen'
    elif c < 100000:
        return 'orange'
    else:
        return 'red'

In [ ]:
center = folium.Map(location=[24.7, 46.73333], zoom_start=6, zoom_control=False, scrollWheelZoom=False, dragging=False, tiles='Stamen Terrain')

In [ ]:
fg = folium.FeatureGroup(name="regions")


In [ ]:
for i in region_coordinates.items():
    
    fg.add_child(folium.Marker(location=[i[1][2], i[1][3]], 
                               popup=folium.Popup(f'{i[1][1]} {i[1][4]} حالة', max_width=len(i[1][1]) * 20), 
                               icon=folium.Icon(color=color(i[1][4]))))
center.add_child(fg)
center

# Choropleth maps
> define a function to change names of regions to match the names in geo file.<br>
> add new columns to the cases data frame for new region names.<br>
> define new data frame for regions_cases to add it as data in Choropleth maps.<br>

In [ ]:
def change_region_name(region):
    if region == "Eastern Region":
        return "Eastern Region"
    elif region == "Ar Riyad":
        return "Riyadh"
    elif region == "Makkah Al Mukarramah":
        return "Mecca"
    elif region == "Aseer":
        return "Asir"
    elif region == "Al Qaseem":
        return "Qassim"
    elif region == "Jazan":
        return "Jazan"
    elif region == "Al Madinah Al Munawwarah":
        return "Medina"
    elif region == "Al Bahah":
        return "Al Bahah"
    elif region == "Hail":
        return "Hail"
    elif region == "Tabuk":
        return "Tabuk"
    elif region == "Najran":
        return "Najran"
    elif region == "Al Jawf":
        return "Al Jouf"
    elif region == "Northern Borders":
        return "Northern Borders"

In [ ]:
cases["new_region_name"] = cases['region'].apply(change_region_name)

In [ ]:
# regions_cases = cases.groupby(['region'], as_index=False).sum()[["region","cases"]]
regions_cases = cases.groupby(['new_region_name'], as_index=False).sum()[["new_region_name","cases"]]

In [ ]:
regions_geo_file = 'SA_regions.json'

m = folium.Map(location=[24, 46], zoom_start=6, scrollWheelZoom=False, dragging=False)

folium.Choropleth(
    geo_data=regions_geo_file,
    name="choropleth",
    data=regions_cases,
    columns=["new_region_name", "cases"],
    key_on="feature.properties.name",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Cases Rate (%)",
).add_to(m)

folium.LayerControl().add_to(m)

m

# Word Cloud
> #### Use `wordcloud` Library to draw total cases per city.

In [ ]:
ksa_map = np.array(Image.open(os.path.join("logos/ksa2.png")))
text = cases.city.value_counts().to_dict()
text = {t.replace(' ', '').replace('ʻ', '').replace("'", "").replace('d͟', 'd').replace('D͟', 'D'): i * 20 for t,i in text.items()}



def transform_format(val):
    if val == 0:
        return 255
    else:
        return val
    

stopwords = set(STOPWORDS)
stopwords.add("said")
    
# Transform your mask into a new one that will work with the function:
transformed_ksa_map = np.ndarray((ksa_map.shape[0],ksa_map.shape[1]), np.int32)

for i in range(len(ksa_map)):
    transformed_ksa_map[i] = list(map(transform_format, ksa_map[i]))

    
# Create a word cloud image
wc = WordCloud(background_color="white", max_words=1000, mask=transformed_ksa_map,
               stopwords=stopwords, contour_width=0, contour_color='firebrick')

# Generate a wordcloud
wc.generate_from_frequencies(text)

# store to file
wc.to_file("logos/ksa3.png")

# show
plt.figure(figsize=[25,12])
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# translate cities to arabic
arabic = {'ArRiyad' : 'الرياض',
 'Jeddah': 'جده',
 'MakkahAlMukarramah': 'مكة المكرمة',
 'AdDammam': 'الدمام',
 'AlMadinahAlMunawwarah': 'المدينة المنورة',
 'AlHufuf': 'الهفوف',
 'AtTaʼIf': 'الطايف',
 'AdHDHahran': 'الظهران',
 'Buraydah': 'بريدة',
 'AlQatif': 'القطيف',
 'AlKhubar': 'الخبر',
 'KhamisMushayt': 'خميس مشيط',
 'Tabuk': 'تبوك',
 'Yanbu': 'ينبع',
 'HaʼIl': 'حائل',
 'AlJubayl': 'الجبيل',
 'Najran': 'نجران',
 'Abha': 'ابها',
 'WadiAdDawasir': 'وادي الدواسر',
 'AlKharj': 'الخرج',
 'AlMubarraz': 'المبرز',
 'Unayzah': 'عنيزة',
 'ArAr': 'عرعر',
 'AlMajmaAh': 'المجمعة',
 'Jazan': 'جازان',
 'Buqayq': 'بقيق',
 'AnNuAyriyyah': 'النعيرية',
 'AdDilam': 'الدلم',
 'HafarAlBatin': 'حفر الباطن',
 'HawtatBaniTamim': 'حوطة بني تميم',
 'AlKhafji': 'الخفجي',
 'AzZulfi': 'الزلفي',
 'Billasmar': 'بللسمر',
 'AdDuwadimi': 'الدوادمي',
 'AlMidhnab': 'المذنب',
 'Biljurashi': 'بلجرشي',
 'AlQunfudhah': 'القنفذة',
 'Bishah': 'بيشة',
 'AlAsyah': 'الاسياح',
 'Sikaka': 'سكاكا',
 'Duruma': 'ضرما',
 'AlBahah': 'الباحة',
 'Khulays': 'خليص',
 'Sharurah': 'شرورة',
 'UyunAlJiwaʼ': 'عين الجواء',
 'QaryatAlUlya': 'قرية العليا',
 'DHahranAlJanub': 'ظهران الجنوب',
 'RiyadAlKhabraʼ': 'رياض الخبراء',
 'AlQurayyat': 'القريات',
 'Afif': 'عفيف',
 'AlJafr': 'الجفر',
 'AlLith': 'الليث',
 'RasTannurah': 'رأس تنورة',
 'AlBadaʼI': 'البدائع',
 'Baysh': 'بيش',
 'RafaIAlJamsh': 'رفائع الجمش',
 'Mahayil': 'محايل عسير',
 'AbuArish': 'ابو عريش',
 'RijalAlMa': 'رجال المع',
 'AlBukayriyyah': 'البكيرية',
 'Mulayjah': 'مليجة',
 'AlMikhwah': 'المخواة',
 'AhadRifaydah': 'احد رفيدة',
 'Khaybar': 'خيبر',
 'AlQuwayIyyah': 'القويعية',
 'AlKhurmah': 'الخرمة',
 'Sabya': 'صبيا',
 'MihdAdhDhahab': 'مهد الذهب',
 'AlKamil': 'الكامل',
 'Samtah': 'صامطة',
 'Adam': 'أضم',
 'ArRass': 'الرس',
 'Safwá': 'صفوى',
 'Rafha': 'رفحاء',
 'AlAqiq': 'العقيق',
 'AlIs': 'العيص',
 'Shaqraʼ': 'شقراء',
 'AdDirIyyah': 'الديرة',
 'AlMudHaylif': 'المظيلف',
 'Umlujj': 'املج',
 'AlHinakiyyah': 'الحناكية',
 'AsSulayyil': 'السليل',
 'AlUyun': 'االعلا',
 'AlHarajah': 'الحرجة',
 'AnNamas': 'النماص',
 'AlMuzahimiyyah': 'المزاحمية',
 'Turayf': 'طريف',
 'Turuba': 'تربه',
 'AdDarb': 'الدرب',
 'Tabarjal': 'طبرجل',
 'AlBashayer': 'البشاير',
 'Fayfa': 'فيفا',
 'AhadAlMusarihah': 'احد المسارحة',
 'AlUla': 'العلا',
 'AlIdabi': 'العيدابي',
 'Huraymilaʼ': 'حريملاء',
 'Qusaybaʼ': 'قصيباء',
 'AtTuwal': 'الطوال',
 'SaratAbidah': 'سراة عبيدة',
 'AlQaysumah': 'القيصومة',
 'AshShinan': 'الشنانة',
 'WadiAlFara': 'وادي الفرع',
 'BadrAlJanub': 'بدر الجنوب',
 'SabtAlAlayah': 'سبت العلايه',
 'Tumayr': 'تمير',
 'Mawqaq': 'موقق',
 'Bariq': 'بارق',
 'AlBijadiyyah': 'البجادية',
 'AlQuwarah': 'القوارة',
 'AlAridah': 'العارضة',
 'AlMajardah': 'المجاردة',
 'Duba': 'ضبا',
 'Bathaʼ': 'بطحاء',
 'Ranyah': 'رنية',
 'UrayIrah': 'عريعرة',
 'AlMuwayh': 'المويه',
 'Rabigh': 'رابغ',
 'HawtatSudayr': 'حوطة سدير',
 'Tabalah': 'تبالة',
 'Tathlith': 'تثليث',
 'Yadamah': 'يدمه',
 'Sajir': 'ساجر',
 'AlMandaq': 'المندق',
 'AlQari': 'القري',
 'DawmatAlJandal': 'دومة الجندل',
 'AlQahmah': 'القحمة',
 'Qilwah': 'قلوة',
 'ArRuwaydah': '',
 'AlGouz': 'القوز',
 'ArRayth': 'الريث',
 'AlBad': '',
 'AlHajrah': 'الحجرة',
 'AshShamli': 'الشملي',
 'Laylá': 'ليلى',
 'AlHaʼIt': 'الحائط',
 'BaqAʼ': 'بقعاء',
 'AlArtawiyyah': 'الارطاوية',
 'UqlatAsSuqur': 'عقلة الصقور',
 'Thurayban': 'ثريبان',
 'AlGhazalah': 'الغزالة',
 'Numarah': 'نمره',
 'AlMaddah': '',
 'Hubuna': 'حبونا',
 'AdDaʼIr': 'الداير',
 'Dariyyah': '',
 'AlBirk': 'البرك',
 'AnNabhaniyyah': 'النبهانية',
 'Khubash': '',
 'Thadiq': 'ثادق',
 'Rumah': 'رماح',
 'UmmAdDawm': 'ام الدوم',
 'AlHamnah': '',
 'AlQara': 'القرى',
 'Haql': 'حقل',
 'AlWajh': 'الوجه',
 'Taymaʼ': 'تيماء',
 'Qiya': 'قيا',
 'ArRayn': '',
 'AlFarshah': 'الفرشه',
 'AlSehen': '',
 'Tanumah': 'تنومه',
 'AlShoBah': '',
 'WadiIbnHashbal': 'وادي بن هشبل',
 'AlHadithahBorderStation': 'الحديثة',
 'AlMuwassam': 'الموسم',
 'AlHarth': 'الحرث',
 'AbuRakah': 'ابو راكه',
 'Damad': 'ضمد',
 'Samiraʼ': 'سميرة',
 'Marat': 'مرات',
 'DHalm': 'ضلم',
 'Farasan': 'فرسان',
 'Badr': 'بدر',
 'Maysan': 'ميسان',
 'Wuthaylan': 'وثيلان',
 'Salwá': 'سلوى',
 'AlMahani': 'المحاني',
 'AlKhasirah': 'الخاصرة',
 'AlHariq': 'الحريق',
 'AlUwayqilah': '',
 'Thuwal': 'ثول',
 'BaniHasan': 'بني حسن',
 'AsSuAyyirah': '',
 'Tarj': 'ترج',
 'AlGhat': 'الغاط',
 'Thar': 'ثار',
 'AsSulaymi': 'السليمي',
 'Shuwaq': 'الشواق',
 'AdhDhibiyyah': '',
 'Sayhat': 'سيهات',
 'AbuAjram': 'أبو عجرم',
 'AlHada': 'الهدا',
 'Nifi': 'نفي',
 'Mayqu': 'ميقوع',
 'JudayyidatArAr': 'جديدة عرعر',
 'AsSufairy': 'الصفيري',
 'Hadda': 'حدة',
 'HazmAlJalamid': 'حزم الجلاميد',
 'Suwayr': 'صوير',
 'ArRuqI': 'الرقه',
 'AbuUrwah': 'ابو عروة',
 'Harad': 'حرض',
 'AsSulaymaniyyah': 'السليمانية',
 'HadadBaniMalik': 'حداد بني ماالك',
 'AlHadban': 'الحدبان',
 'Samudah': 'سامودة'}


In [ ]:
# make dictionary for arabic city and cases
cases_arabic_city = {}
cases_arabic_only_city = {}
for i in text:
    cases_arabic_city[i] = [i, arabic[i], text[i]]
    cases_arabic_only_city[arabic[i]] = text[i]


In [ ]:
# -*- coding: utf-8 -*-

ksa_map = np.array(Image.open(os.path.join("logos/ksa2.png")))
text_1 = cases_arabic_only_city
text_1 = {get_display(arabic_reshaper.reshape(t)): i * 20 for t,i in text_1.items()}




# Transform your mask into a new one that will work with the function:
transformed_ksa_map = np.ndarray((ksa_map.shape[0],ksa_map.shape[1]), np.int32)

for i in range(len(ksa_map)):
    transformed_ksa_map[i] = list(map(transform_format, ksa_map[i]))

    
# Create a word cloud image
wc = WordCloud(background_color="white", max_words=1000, mask=transformed_ksa_map,
               stopwords=stopwords, contour_width=0, contour_color='firebrick', font_path='logos/NotoNaskhArabic-Regular.ttf')

# Generate a wordcloud
wc.generate_from_frequencies(text_1)

# store to file
wc.to_file("logos/ksa4.png")

# show
plt.figure(figsize=[25,12])
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()